# Analysing Dataset

This notebook is dedicated to datasets analysis. 

Here we will concentrate on connecting our datasets into one dataframe that will later be used to extract meaningful information that can help us answer our research questions. 

In addition, data cleaning is performed where necessary since we will not use all the data provided. The goal of this notebook is to make data as easy as possible to use for future plotting and data story writing.

So, let's dive into our data!

In [1]:
import pandas as pd
import pandas_profiling
import requests
from functional import pseq, seq
import pathlib
import os
import json
import time
import datetime
from json import load, JSONDecodeError

In [2]:
directory_liar_dataset = "../data/liar_dataset"
directory_statements = f"{directory_liar_dataset}/statements"
directory_visualizations = "../docs/data_insight"
directory_election_results = "../data/election_results"

This is just the LIAR dataset:

In [3]:
df = pd.concat([pd.read_csv(f"{directory_liar_dataset}/{part}.tsv", sep='\t', header=None) for part in ['train', 'valid']])
df.columns = ['statement_id', 'label', 'statement', 'subject', 'speaker', 'speakers_job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context']
# removing .json and getting just ID
df.statement_id = df.statement_id.apply(lambda x: x[:-5])  

In [336]:
df.head(2)

,statement_id,label,statement,subject,speaker,speakers_job_title,state_info,party_affiliation,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context,label_as_nb
0,2635,false,Says the Annies List political group supports third-trimester abortions on demand.,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,8
1,10540,half-true,When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration.,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,4


And now, let's read the data regarding corresponding statements and merge useful information from statements with the liar dataset:

In [6]:
def extract_information(res):
    """ Extracting information from JSON statements
    
    Parameters
    ----------
    res: dict
        The dictionary containing collected information of one statement, the statement that is collected using web scrapping.
    
    Returns
    -------
    dict
        The dictionary only with the data we need for our data analysis.
    """
    try:
        author = res['author']

        try:
            if len(author) > 0:
                author = author[0]['name_slug']
            else:
                author = None
        except Exception:
            print(author)

        return {'author_name_slug': author,
                'ruling_date':  res['ruling_date'],
                'statement_date' :res['statement_date'],
                'speaker_current_job': res['speaker']['current_job'],
                'speaker_first_name': res['speaker']['first_name'],
                'speaker_last_name': res['speaker']['last_name'],
                'speaker_home_state': res['speaker']['home_state'],
                'statement_id': res['id']
               }
    except KeyError:
        return {}

In [7]:
def safe_json_read(f):
    """ Safely reading the JSON file
    
    Parameters
    ----------
    f: str
        File name of JSON
    
    Returns
    -------
    dict
        Dictionary with data read from JSON file
    
    Rasises
    -------
    JSONDecodeError: If JSON file doesn't have JSON format.
    """
    try:
        with open(f, 'r') as fc:
            return load(fc)
    except JSONDecodeError as de:
        print(f"File {f} is empty or something...")
        return {}

In [8]:
additional_information = seq(pathlib.Path(directory_statements).iterdir()).map(safe_json_read)\
                               .filter(lambda x: len(x) > 0)\
                               .map(extract_information)\
                               .to_pandas()

additional_information['statement_date'] = pd.to_datetime(additional_information['statement_date'])
additional_information.head()

File ..\data\liar_dataset\statements\5355.json is empty or something...
File ..\data\liar_dataset\statements\9.json is empty or something...


,author_name_slug,ruling_date,speaker_current_job,speaker_first_name,speaker_home_state,speaker_last_name,statement_date,statement_id
0,meghan-ashford-grooms,2010-01-12T15:52:21,,Barbara Ann,,Radnofsky,2009-10-22,1
1,jody-kyle,2007-10-03T00:00:00,author,Mike,Arkansas,Huckabee,2007-09-21,100
2,bill-adair,2009-04-30T18:20:28,Congresswoman,Michele,Minnesota,Bachmann,2009-04-27,1000
3,w-gardner-selby,2014-10-02T14:07:32,Pharmacist,Leticia,Texas,Van de Putte,2014-09-29,10000
4,tom-kertscher,2014-10-02T13:15:42,,,Wisconsin,Wisconsin Professional Police Association,2014-09-25,10001


In [9]:
def label_to_nb(l): 
    """ Converting label to number
    
    Parameters
    ----------
    l: str
        Label of the lie, can be 'true', 'mostly-true', 'half-true', 'barely-true', 'false', 'pants-fire'
        
    Returns
    -------
    int
        Number in range from 0 to 5. We still need to think about lie representation.
    
    ToDos
    -----
    - think about this, this will give the false-hoods more weight
    """
    return ['true', 'mostly-true', 'half-true', 'barely-true', 'false', 'pants-fire'].index(l)

df['label_as_nb'] = df['label'].apply(label_to_nb) * 2 

In [10]:
df['statement_id'] = pd.to_numeric(df['statement_id'])
lies = df.merge(additional_information, on='statement_id', how='left')

Let's just see what we have here:

In [11]:
lies.loc[lies['speaker'] == 'barack-obama', ]['pants_on_fire_counts'].value_counts()

9.0    549
Name: pants_on_fire_counts, dtype: int64

Here we can see that Barack Obama had 549 statements labeled with _pants on fire_.

In [12]:
lies[lies['speakers_job_title'].str.contains('County') == True].shape

(265, 22)

In [13]:
lies['statement_date'].describe()

count                   11521
unique                   2832
top       2011-01-25 00:00:00
freq                       22
first     1995-04-01 00:00:00
last      2016-12-11 00:00:00
Name: statement_date, dtype: object

Above, we can see that statements range from 1995 to 2016.

Now, let's do some profiling to get some more insights:

In [14]:
pandas_profiling.ProfileReport(lies)

Number of variables,22
Number of observations,11524
Total Missing (%),2.3%
Total size in memory,2.0 MiB
Average record size in memory,184.0 B
Numeric,4
Categorical,14
Boolean,0
Date,1
Text (Unique),0
Rejected,3


# Federal Election Results

We have another dataset that we will explore and merge to our LIAR dataset in order to get some more insight into data. This one is regarding election results.

In [15]:
pd.options.display.max_colwidth = 300
pd.options.display.max_columns = 300

In [16]:
from itertools import product
from functools import reduce

In [17]:
def add_ending(f):
    """ File ending depending on a year
    
    Parameters
    ----------
    f: str
        Name of the file
    
    ToDos:
    - do 2012 it's a special snowflake
    """
    if '2016' in f:
        return f"{f}x"
    else:
        return f


election_files = [(add_ending(f'{directory_election_results}/federalelections{year}.xls'), year) for year in [2014, 2016]]

Now, let's prepare some data for viewing:

In [18]:
election_results_cols_of_interest = ['CANDIDATE NAME', 'PRIMARY VOTES', 'PRIMARY %']

def fix_columns_election_results(df, year, type_):
    df = df.loc[:, election_results_cols_of_interest]
    df[f'primary_votes_{type_.lower()}_{year}'] = df['PRIMARY VOTES']
    df[f'primary_votes_{type_.lower()}_{year}_pct'] = df['PRIMARY %']
    return df.drop(columns=['PRIMARY VOTES', 'PRIMARY %'])


def get_only_voting_results(df):
    return df.loc[df['CANDIDATE NAME'].notna() & df['PRIMARY VOTES'].notna() & df['CANDIDATE NAME'].ne('Scattered') & df['CANDIDATE NAME'].ne('All Others'), :]


def prep_election_results(df, year, type_):
    return fix_columns_election_results(get_only_voting_results(df), year, type_)

In [19]:
election_results = [prep_election_results(pd.read_excel(f, sheet_name=f'{year} US {type_} Results by State'), year, type_) for (f, year), type_ in product(election_files, ['Senate', 'House'])]
election_results = reduce(lambda acc, el: pd.merge(acc, el, on='CANDIDATE NAME', how='outer'), election_results)

In [20]:
# yeah ... let's see how many we can join. the one letter endings might be a problem
election_results['CANDIDATE NAME'].value_counts()

Reed, Thomas W., II                         12
Katko, John M.                              12
Tonko, Paul D.                              12
Maloney, Sean Patrick                       12
Collins, Chris                              12
Assini, Mark W.                              8
Stefanik, Elise M.                           8
Slaughter, Louise M.                         6
Crowley, Joseph                              6
Nadler, Jerrold L.                           6
Lowey, Nita M.                               6
Zeldin, Lee M.                               6
Engel, Eliot L.                              6
Higgins, Brian                               6
Jeffries, Hakeem S.                          4
Kuster, Ann McLane                           4
Faso, John J.                                4
Rubens, Jim                                  4
Donovan, Dan                                 4
Schumer, Charles E.                          4
Martin, Andy                                 4
Clarke, Yvett

In [21]:
# we are only interest in people and they have a first name
lies = lies.loc[lies['speaker_first_name'].notnull(), :]

In [22]:
# to aggregate the statements
lies['statement_year'] = lies['statement_date'].dt.year

# for the merging
lies['speaker_full_name'] = lies['speaker_last_name'] + ', ' + lies['speaker_first_name']

### Cleaning job titles

In [23]:
# todo expand this and check this! this is just a quick and dirty fix
# is it really houseman? probably not...
_job_titles_of_interest = [('senat', 'senator'), ('governor', None), ('congress', 'congressman'), ('mayor', None), ('president', None), ('house', 'houseman'), ('rep', 'houseman')]
job_titles_of_interest = [out if out is not None else j for j, out in _job_titles_of_interest]

def cleaned_job_title(jt):
    jt = str(jt).lower()
    
    for j, out in _job_titles_of_interest:
        if j in jt:
            return out if out is not None else j
    else:
        return jt

lies['speakers_job_title_cleaned'] = lies['speakers_job_title'].apply(cleaned_job_title)

In [24]:
_t = lies.merge(election_results, left_on='speaker_full_name', right_on='CANDIDATE NAME', how='outer')

In [25]:
print(f"found election results for {_t['CANDIDATE NAME'].notnull().sum()} ({_t['CANDIDATE NAME'].notnull().mean()}%) people")

found election results for 4952 (0.328403740301081%) people


In [26]:
votes_cols = [c for c in _t.columns if 'votes' in c]
useful_idx = reduce(lambda acc, el: acc | el, [_t[c].notnull() for c in votes_cols]) & _t['speaker'].notnull() 

print(f"found useful results for {useful_idx.sum()} people")

columns_of_interest = ['label', 'label_as_nb', 'subject', 'speaker', 'speakers_job_title_cleaned', 'state_info', 'party_affiliation', 'context', 'statement_date'] + votes_cols
_t.loc[useful_idx, columns_of_interest]

found useful results for 1461 people


,label,label_as_nb,subject,speaker,speakers_job_title_cleaned,state_info,party_affiliation,context,statement_date,primary_votes_senate_2014,primary_votes_senate_2014_pct,primary_votes_house_2014,primary_votes_house_2014_pct,primary_votes_senate_2016,primary_votes_senate_2016_pct,primary_votes_house_2016,primary_votes_house_2016_pct
1115,half-true,4.0,"financial-regulation,foreign-policy,water",gwen-moore,houseman,Wisconsin,democrat,a congressional hearing,2016-04-27,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1116,mostly-true,2.0,"crime,criminal-justice,women",gwen-moore,houseman,Wisconsin,democrat,a letter to congressional leadership,2012-12-11,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1117,barely-true,6.0,"city-government,civil-rights,criminal-justice,public-safety,state-budget",gwen-moore,houseman,Wisconsin,democrat,comments to reporters,2015-08-26,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1118,true,0.0,"city-government,infrastructure,transportation",gwen-moore,houseman,Wisconsin,democrat,a newspaper opinion piece,2015-01-14,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1119,false,8.0,"children,crime,criminal-justice,sexuality",gwen-moore,houseman,Wisconsin,democrat,remarks on the House floor,2014-05-29,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1120,pants-fire,10.0,"corrections-and-updates,health-care,women",gwen-moore,houseman,Wisconsin,democrat,tweets,2011-12-27,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1121,true,0.0,"civil-rights,crime,criminal-justice,legal-issues",gwen-moore,houseman,Wisconsin,democrat,an interview,2016-11-18,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1122,mostly-true,2.0,"crime,guns,terrorism,transportation",gwen-moore,houseman,Wisconsin,democrat,a tweet,2016-06-23,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1123,barely-true,6.0,"economy,health-care,medicaid,medicare",gwen-moore,houseman,Wisconsin,democrat,a radio interview,2014-11-11,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1124,false,8.0,"labor,state-budget,state-finances",gwen-moore,houseman,Wisconsin,democrat,a speech at a state Democratic Party dinner,2011-04-30,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN


In [31]:
_t.loc[useful_idx, 'speakers_job_title_cleaned'].value_counts()

senator                                                                                                 580
houseman                                                                                                353
milwaukee county executive                                                                              166
congressman                                                                                             111
nan                                                                                                     101
governor                                                                                                 41
small business owner                                                                                     13
businessman                                                                                              13
retired social worker                                                                                     8
conservative activist       

In [32]:
_t.loc[_t['speakers_job_title_cleaned'].isin(job_titles_of_interest) & useful_idx, columns_of_interest]

,label,label_as_nb,subject,speaker,speakers_job_title_cleaned,state_info,party_affiliation,context,statement_date,primary_votes_senate_2014,primary_votes_senate_2014_pct,primary_votes_house_2014,primary_votes_house_2014_pct,primary_votes_senate_2016,primary_votes_senate_2016_pct,primary_votes_house_2016,primary_votes_house_2016_pct
1115,half-true,4.0,"financial-regulation,foreign-policy,water",gwen-moore,houseman,Wisconsin,democrat,a congressional hearing,2016-04-27,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1116,mostly-true,2.0,"crime,criminal-justice,women",gwen-moore,houseman,Wisconsin,democrat,a letter to congressional leadership,2012-12-11,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1117,barely-true,6.0,"city-government,civil-rights,criminal-justice,public-safety,state-budget",gwen-moore,houseman,Wisconsin,democrat,comments to reporters,2015-08-26,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1118,true,0.0,"city-government,infrastructure,transportation",gwen-moore,houseman,Wisconsin,democrat,a newspaper opinion piece,2015-01-14,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1119,false,8.0,"children,crime,criminal-justice,sexuality",gwen-moore,houseman,Wisconsin,democrat,remarks on the House floor,2014-05-29,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1120,pants-fire,10.0,"corrections-and-updates,health-care,women",gwen-moore,houseman,Wisconsin,democrat,tweets,2011-12-27,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1121,true,0.0,"civil-rights,crime,criminal-justice,legal-issues",gwen-moore,houseman,Wisconsin,democrat,an interview,2016-11-18,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1122,mostly-true,2.0,"crime,guns,terrorism,transportation",gwen-moore,houseman,Wisconsin,democrat,a tweet,2016-06-23,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1123,barely-true,6.0,"economy,health-care,medicaid,medicare",gwen-moore,houseman,Wisconsin,democrat,a radio interview,2014-11-11,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN
1124,false,8.0,"labor,state-budget,state-finances",gwen-moore,houseman,Wisconsin,democrat,a speech at a state Democratic Party dinner,2011-04-30,NaN,NaN,52413,0.709127,NaN,NaN,NaN,NaN


Now, our dataframe looks like this:

In [33]:
_t.head(1)

,statement_id,label,statement,subject,speaker,speakers_job_title,state_info,party_affiliation,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context,label_as_nb,author_name_slug,ruling_date,speaker_current_job,speaker_first_name,speaker_home_state,speaker_last_name,statement_date,statement_year,speaker_full_name,speakers_job_title_cleaned,CANDIDATE NAME,primary_votes_senate_2014,primary_votes_senate_2014_pct,primary_votes_house_2014,primary_votes_house_2014_pct,primary_votes_senate_2016,primary_votes_senate_2016_pct,primary_votes_house_2016,primary_votes_house_2016_pct
0,10540.0,half-true,When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration.,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,4.0,sean-gorman,2015-02-23T00:00:00,State delegate,Scott,Virginia,Surovell,2015-02-04,2015.0,"Surovell, Scott",state delegate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# DATA SET COMPLETE

At this point, we collected all the columns we need. Let's see how we can clean them:

In [34]:
median_speaker_value = _t.groupby(['statement_year', 'speaker'])['label_as_nb'].median().reset_index()

In [35]:
median_speaker_value[median_speaker_value['statement_year'] == 2016]

,statement_year,speaker,label_as_nb
4334,2016.0,18-percent-american-public,0.0
4335,2016.0,Jack_Graham,8.0
4336,2016.0,aclu-north-carolina,0.0
4337,2016.0,actionaid-k,3.0
4338,2016.0,afscme,4.0
4339,2016.0,afscme-people,2.0
4340,2016.0,al-gore,4.0
4341,2016.0,alan-grayson,3.0
4342,2016.0,alcee-hastings,10.0
4343,2016.0,alex-castellanos,6.0


### Non-People Speakers Handling

Removing non-people (_tweets, facebook posts, etc._) from the dataset:

In [146]:
from nltk import download
download('punkt')
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
from nltk import sent_tokenize
from collections import Counter

model = 'nlp/stanford-ner-2018-10-16/classifiers/english.all.3class.distsim.crf.ser.gz'
jar = 'nlp/stanford-ner-2018-10-16/stanford-ner-3.9.2.jar'
st = StanfordNERTagger(model, jar, encoding='utf-8')

def get_tag(speaker):
    ner_tag = 0
    if type(speaker) == str:
        full_speaker_name = speaker.replace("-", " ").title()

        for sent in sent_tokenize(full_speaker_name):
            tokens = word_tokenize(sent)
            tags = st.tag(tokens)
            
        ner_tag= Counter(dict(tags).values()).most_common(1)[0][0]
        print(tags, " --> ", ner_tag)
    return ner_tag
    

# just to see if/how it works
word = "Twitter-Post-Anna"
get_ner_tag(word)

full_speaker_name = "Barack-Obama"
get_ner_tag(full_speaker_name)

full_speaker_name = 0
get_ner_tag(full_speaker_name)


[nltk_data] Downloading package punkt to C:\Users\Jelena
[nltk_data]     Banjac\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[('Twitter', 'ORGANIZATION'), ('Post', 'ORGANIZATION'), ('Anna', 'PERSON')]  -->  ORGANIZATION
[('Barack', 'PERSON'), ('Obama', 'PERSON')]  -->  PERSON


0

In [199]:
import os.path

file_path = 'nlp/speaker_tags.json'

if not os.path.exists(file_path):
    print(f"Total number of values to classify: {len(_t['speaker'].value_counts().index)}")

    words_with_tags = {}
    for word in _t['speaker'].value_counts().index:
        words_with_tags[word] = get_tag(word)
    
    # save tags, since it took ~3h to tag all 3214 unique speakers
    with open(file_path, 'w') as fp:
        json.dump(words_with_tags, fp, indent=4)
else:
    with open(file_path, 'r') as f:
        words_with_tags = json.load(f)
    print(f"Total number of classified values (from file): {len(words_with_tags)}")

Total number of classified values (from file): 3124


In [200]:
_t["speaker_tag"] = _t.apply(lambda row: words_with_tags[row['speaker']] if not pd.isnull(row['speaker']) else row['speaker'], axis=1)
_t[['speaker','speaker_tag']].drop_duplicates()

,speaker,speaker_tag
0,scott-surovell,PERSON
1,barack-obama,PERSON
370,bobby-jindal,PERSON
623,charlie-crist,PERSON
704,robin-vos,PERSON
725,oregon-lottery,PERSON
727,duey-stroebel,PERSON
728,robert-menendez,PERSON
739,bernie-s,PERSON
838,mitt-romney,PERSON


Seems good, so now let's remove non-people from the dataset:

In [201]:
_t.shape

(9803, 37)

In [202]:
_t[_t['speaker_tag'] == "PERSON"].shape

(9803, 37)

We see that we will remove ~5000 statements which are made by (speaker) _Twitter, Facebook, Blog post, Republican Party Texas, etc._

In [203]:
# removing non-people statements
_t = _t[_t['speaker_tag'] == "PERSON"]

### Clean-up Context

In [228]:
_t["context"].value_counts().index.values

array(['an interview', 'a speech', 'a news release', ...,
       'an immigration forum', 'Atlanta City Council meeting',
       'an episode of The Rachel Maddow Show.'], dtype=object)

It would be good to try using the tool that would extract keywords from these phases. Let's use NLTK Rake:

In [327]:
from rake_nltk import Rake, Metric
from collections import Counter

debug = False

def do_keyword_extraction(words):
    if debug: print("---\n", words)
        
    rake_all = Rake()
    rake_all.extract_keywords_from_sentences(_t["context"].value_counts().index.values)

    word_degrees = dict(rake_all.get_word_degrees())
    
    r = Rake()
    r.extract_keywords_from_text(words)

    keywords = dict(r.get_word_degrees())
    
    if debug: print(keywords)
        
    for k, v in keywords.items():
        keywords[k] = word_degrees[k]
    
    if debug: print(keywords)

    return Counter(keywords).most_common(1)[0]

In [328]:
# try to see how it works
text_to_process = "a television interview"
do_keyword_extraction("an interview")
do_keyword_extraction("a television interview")
do_keyword_extraction("a TV interview")

('interview', 158)

In [329]:
_t["context_tag"] = _t.apply(lambda row: do_keyword_extraction(row['context']) if not pd.isnull(row['context']) else row['context'], axis=1)

In [334]:
context_tags = _t[['context','context_tag']]['context_tag'].value_counts()
print(f"Number of different context tags is {len(context_tags)}")
context_tags

Number of different context tags is 271


(interview, 158)       1268
(speech, 175)           877
(debate, 437)           874
(news, 329)             833
(campaign, 406)         729
(press, 228)            656
(ad, 126)               355
(radio, 198)            267
(state, 183)            231
(post, 97)              205
(show, 227)             204
(house, 222)            194
(tweet, 16)             181
(senate, 229)           154
(column, 53)            147
(meeting, 149)          119
(article, 78)           112
(new, 205)               95
(statement, 69)          91
(video, 92)              84
(letter, 33)             81
(email, 48)              81
(website, 72)            75
(ed, 45)                 74
(abc, 23)                70
(texas, 168)             65
(forum, 78)              63
(hearing, 149)           56
(journal, 71)            48
(rally, 45)              46
                       ... 
(reporter, 11)            1
(virginia, 1)             1
(co, 5)                   1
(filibuster, 6)           1
(teleconference, 1) 

We see that the number of context tags is 271, which is a preatty big number. Let's consider decreasing this number and make smaller groups.

In [ ]:
# TODO: make smaller context groups, ideally around 10

In [ ]:
# TODO: implement tagging on the cleaned jobs as well

In [ ]:
# TODO: plot the answers from research questions we have

## [misc] One statement content visualization

Let's analyse first row, statement with id `1`. What is the information we get there?

In [190]:
sid = 1.0

In [191]:
_t[_t.statement_id == sid]

,statement_id,label,statement,subject,speaker,speakers_job_title,state_info,party_affiliation,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context,label_as_nb,author_name_slug,ruling_date,speaker_current_job,speaker_first_name,speaker_home_state,speaker_last_name,statement_date,statement_year,speaker_full_name,speakers_job_title_cleaned,CANDIDATE NAME,primary_votes_senate_2014,primary_votes_senate_2014_pct,primary_votes_house_2014,primary_votes_house_2014_pct,primary_votes_senate_2016,primary_votes_senate_2016_pct,primary_votes_house_2016,primary_votes_house_2016_pct,ner_lbl,ner_tag
11184,1.0,pants-fire,The attorney general requires that rape victims pay for the rape kit.,"crime,women",barbara-ann-radnofsky,NaN,NaN,democrat,0.0,0.0,0.0,0.0,1.0,in a Web site video,10.0,meghan-ashford-grooms,2010-01-12T15:52:21,,Barbara Ann,,Radnofsky,2009-10-22,2009.0,"Radnofsky, Barbara Ann",nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PERSON


In [337]:
with open(f"{directory_statements}/{int(sid)}.json", "r") as f:
    data = json.load(f)

Just to visualize JSON hierarchy, run the following cell:

In [ ]:
def go_further(dic, name):
    dict_vis = {"name": name, "children": []}
    for k, v in dic.items():
        if type(v) == str:
            new_el = {"name": k}
        elif type(v) == list:
            if len(v) > 0:
                new_el = go_further(v[0], k)
        elif type(v) == dict:
            new_el = go_further(v, k)
        else:
            new_el = {"name": k}
        dict_vis["children"].append(new_el)
        
    return dict_vis

my_dict = go_further(data, name="statement_info")

with open(f"{directory_visualizations}/data.json", "w") as f:
    json.dump(my_dict, f)

print(f"Checkout visualization by: \n1) cd ../visualizations \n2) python -m http.server \n3) in browser, open: http://localhost:8000/")